In [2]:
import numpy as np
from numpy import load

with load('/home/admin/rnd_test/FloBaRoID/model/youbot_arm_only.urdf.trajectory.npz')as data:
    q= data['q']

In [3]:
data = np.load('/home/admin/rnd_test/FloBaRoID/model/youbot_arm_only.urdf.trajectory.npz')
print(data['a'])
print("\n")
print(data['b'])
print(data['q'])
print(data['wf'])
print(data['nf'])
print(data['use_deg'])
print(data['static'])

[[-0.01843954 -1.35100414 -1.38460642 -1.49999526]
 [ 1.47889483 -0.68627869 -1.49898959  0.62888016]
 [-0.87304354  1.49305501  0.01446198  1.32713111]
 [-0.80310965 -0.27268255  0.20038992 -0.08897228]
 [ 0.32868034  1.47174043  0.3566105  -0.43804508]]


[[ 0.55884867  1.49311535  0.32678246  0.36818675]
 [ 1.32197464  0.23963626  1.49311535 -1.49976685]
 [-0.06066548  0.60247197 -0.8886058  -0.0830268 ]
 [-0.92282281  1.49311535  1.46462059  1.37916413]
 [-0.47232994  1.49311535 -1.49944466  1.14680166]]
[ 1.61959383  1.69281924 -0.47206474  2.86613349  1.98916314]
0.749359503834
[4 4 4 4 4]
0
False


In [4]:
a=np.array([[-0.35707, 0.21389, 0.43313, 0.05424], [-0.37936, 0.19053, 0.54725, 0.53816], [0.37673, 0.00059, -0.39666, 0.50074], [0.8947, 1.5, -0.19924, 0.22437], [-0.37039, 0.33709, -0.30782, -0.96412]])
b=np.array([[0.80867, 1.06223, -0.18519, 0.09188], [-0.11051, 0.211, 0.36583, 0.36593], [0.25166, 1.25223, 1.5, 0.98509], [0.18492, -1.4371, 0.98988, -0.47763], [-0.90683, 0.5881, 0.84402, 1.47759]])
q=np.array([0.5, 0.5, -0.5, 0.5, 0.5])
wf=1.27555600367
nf=np.array([4, 4, 4 ,4 ,4])
static=data['static']
use_deg=data['use_deg']

np.savez('try_traj_1.npz', a=a, b=b,nf=nf,q=q,static=static,use_deg=use_deg,wf=wf)
wf 0.99838078886
a [[-0.77756, -1.49786, -1.21413, 1.49474], [1.05098, 0.11417, 0.32843, 0.69859], [0.44867, -0.54111, -0.56826, -0.09758], [-0.997, 0.27821, 0.24104, 1.49474], [-1.31575, -1.49721, 0.79481, 1.49474]]
b [[0.16809, -1.49951, -0.99636, -1.4835], [0.31666, -0.24893, 1.28017, 0.38497], [-0.42263, 1.42245, 1.49431, 1.40996], [-0.88464, -0.71777, -1.33568, 1.47738], [-0.93205, 1.10828, -0.0538, -0.65378]]
q [0.8788, 0.3643, -0.52153, 0.49884, 0.49383]


SyntaxError: invalid syntax (<ipython-input-4-2eb100e79422>, line 10)

In [5]:
class OscillationGenerator(object):
    def __init__(self, w_f, a, b, q0, nf, use_deg):
        '''
        generate periodic oscillation from fourier series (Swevers, 1997)

        - w_f is the global pulsation (frequency is w_f / 2pi)
        - a and b are (arrays of) amplitudes of the sine/cosine
          functions for each joint
        - q0 is the joint angle offset (center of pulsation)
        - nf is the desired amount of coefficients for this fourier series
        '''
        self.w_f = float(w_f)
        self.a = a
        self.b = b
        self.use_deg = use_deg
        self.q0 = float(q0)
        if use_deg:
            self.q0 = np.deg2rad(self.q0)
        self.nf = nf

    def getAngle(self, t):
        #- t is the current time
        q = 0.0
        for l in range(1, self.nf+1):
            q += (self.a[l-1]/(self.w_f*l))*np.sin(self.w_f*l*t) - \
                 (self.b[l-1]/(self.w_f*l))*np.cos(self.w_f*l*t)
          
        q += self.nf*self.q0
        
        if self.use_deg:
            q = np.rad2deg(q)
        
        return q

    def getVelocity(self, t):
        dq = 0.0
        for l in range(1, self.nf+1):
            dq += self.a[l-1]*np.cos(self.w_f*l*t) + \
                  self.b[l-1]*np.sin(self.w_f*l*t)
        if self.use_deg:
            dq = np.rad2deg(dq)
        return dq

    def getAcceleration(self, t):
        ddq = 0.0
        for l in range(1, self.nf+1):
            ddq += -self.a[l-1]*self.w_f*l*np.sin(self.w_f*l*t) + \
                    self.b[l-1]*self.w_f*l*np.cos(self.w_f*l*t)
        if self.use_deg:
            ddq = np.rad2deg(ddq)
        return ddq
    
class Trajectory(object):
    ''' base trajectory class '''
    def getAngle(self, dof):
        raise NotImplementedError()

    def getVelocity(self, dof):
        raise NotImplementedError()

    def getAcceleration(self, dof):
        raise NotImplementedError()

    def getPeriodLength(self):
        raise NotImplementedError()

    def setTime(self, time):
        raise NotImplementedError()

    def wait_for_zero_vel(self, t_elapsed):
        raise NotImplementedError()

In [6]:
class PulsedTrajectory(Trajectory):
    ''' pulsating trajectory generator for one joint using fourier series from
        Swevers, Gansemann (1997). Gives values for one time instant (at the current
        internal time value)
    '''
    def __init__(self, dofs, use_deg=False):
        # type: (List, bool) -> None
        self.dofs = dofs
        self.oscillators = list()  # type: List[OscillationGenerator]
        self.use_deg = use_deg
        self.w_f_global = 1.0

   
    def initWithParams(self, a, b, q, nf, wf=None):
        ''' init with given params
            a - list of dof coefficients a
            b - list of dof coefficients b
            q - list of dof coefficients q_0
            nf - list of dof coefficients n_f
            (also see docstring of OscillationGenerator)
        '''

        if len(nf) != self.dofs or len(q) != self.dofs:
            raise Exception("Need DOFs many values for nf and q!")

        #for i in nf:
        #    if not ( len(a) == i and len(b) == i):
        #        raise Exception("Need nf many values in each parameter array value!")

        self.a = a
        self.b = b
        self.q = []
        self.nf = nf
        if wf:
            self.w_f_global = wf

        self.oscillators = list()
        for i in range(0, self.dofs):
            self.oscillators.append(OscillationGenerator(w_f = self.w_f_global, a = np.array(a[i]),
                                                         b = np.array(b[i]), q0 = q[i], nf = nf[i], use_deg = self.use_deg
                                                        ))
        return self

    def getAngle(self, dof):
        """ get angle at current time for joint dof """
        return self.oscillators[dof].getAngle(self.time)

    def getVelocity(self, dof):
        """ get velocity at current time for joint dof """
        return self.oscillators[dof].getVelocity(self.time)

    def getAcceleration(self, dof):
        """ get acceleration at current time for joint dof """
        return self.oscillators[dof].getAcceleration(self.time)

    def getPeriodLength(self):
        ''' get the period length of the oscillation in seconds '''
        return 2 * np.pi / self.w_f_global

    def setTime(self, time):
        '''set current time in seconds'''
        self.time = time

    def wait_for_zero_vel(self, t_elapsed):
        self.setTime(t_elapsed)
        if self.use_deg: thresh = 5.0
        else: thresh = np.deg2rad(5.0)
        return abs(self.getVelocity(0)) < thresh

In [7]:
from identification.data import Data
from identification.model import Model

def simulateTrajectory(trajectory, config):
    # type: (Dict, Trajectory, Model, np._ArrayLike) -> Tuple[Dict, Data]
    # generate data arrays for simulation and regressor building
    

   
    dofs=5
    
    trajectory_data = {}   # type: Dict[str, Union[List, np._ArrayLike]]
    trajectory_data['target_positions'] = []
    trajectory_data['target_velocities'] = []
    trajectory_data['target_accelerations'] = []
    trajectory_data['torques'] = []
    trajectory_data['times'] = []

    freq = 200
    for t in range(0, int(trajectory.getPeriodLength()*freq)):
        trajectory.setTime(t/freq)
        q = np.array([trajectory.getAngle(d) for d in range(dofs)])
        trajectory_data['target_positions'].append(q)
        #f.write("%s\n" % q)
        
        qdot = np.array([trajectory.getVelocity(d) for d in range(dofs)])
        trajectory_data['target_velocities'].append(qdot)

        qddot = np.array([trajectory.getAcceleration(d) for d in range(dofs)])
        trajectory_data['target_accelerations'].append(qddot)

        trajectory_data['times'].append(t/freq)
        trajectory_data['torques'].append(np.zeros(dofs))
   
    num_samples = len(trajectory_data['times'])
    
    #convert lists to numpy arrays
    trajectory_data['target_positions'] = np.array(trajectory_data['target_positions'])
    trajectory_data['positions'] = trajectory_data['target_positions']
    trajectory_data['target_velocities'] = np.array(trajectory_data['target_velocities'])
    trajectory_data['velocities'] = trajectory_data['target_velocities']
    trajectory_data['target_accelerations'] = np.array(trajectory_data['target_accelerations'])
    trajectory_data['accelerations'] = trajectory_data['target_accelerations']
    trajectory_data['torques'] = np.array(trajectory_data['torques'])
    trajectory_data['times'] = np.array(trajectory_data['times'])
    trajectory_data['measured_frequency'] = freq
    trajectory_data['base_velocity'] = np.zeros( (num_samples, 6) )
    trajectory_data['base_acceleration'] = np.zeros( (num_samples, 6) )

    trajectory_data['base_rpy'] = np.zeros( (num_samples, 3) )
    
    data = Data(config)
    data.init_from_data(trajectory_data)
    model.computeRegressors(data)
    
    return trajectory_data

In [8]:
'''traj_file = 'try_traj_1.npz'
dof=5       
 #load from file
try:
    # replay optimized trajectory if found
    tf = np.load(traj_file, encoding='latin1')
    # proper trajectory
    trajectory = PulsedTrajectory(dof, use_deg=tf['use_deg'])
    trajectory.initWithParams(tf['a'], tf['b'], tf['q'], tf['nf'], tf['wf'])
    print("using trajectory from file {}".format(traj_file))
except IOError:
        print("No trajectory file found, can't excite ({})!".format(traj_file))
        sys.exit(1)

        
simulateTrajectory(trajectory)
    #- [0.0,5.89921]
    #- [0.0,2.70526] 
    #- [-5.18363,0.0]
    #- [0.0,3.57792]
    #- [0.0,5.75959]'''

'traj_file = \'try_traj_1.npz\'\ndof=5       \n #load from file\ntry:\n    # replay optimized trajectory if found\n    tf = np.load(traj_file, encoding=\'latin1\')\n    # proper trajectory\n    trajectory = PulsedTrajectory(dof, use_deg=tf[\'use_deg\'])\n    trajectory.initWithParams(tf[\'a\'], tf[\'b\'], tf[\'q\'], tf[\'nf\'], tf[\'wf\'])\n    print("using trajectory from file {}".format(traj_file))\nexcept IOError:\n        print("No trajectory file found, can\'t excite ({})!".format(traj_file))\n        sys.exit(1)\n\n        \nsimulateTrajectory(trajectory)\n    #- [0.0,5.89921]\n    #- [0.0,2.70526] \n    #- [-5.18363,0.0]\n    #- [0.0,3.57792]\n    #- [0.0,5.75959]'

In [9]:
dofs = 5
nf = 4

wf_min = 0.01
wf_max = 2.0
wf_init = 0.1

qmin = [-0.5]*dofs
qmax = [0.5]*dofs
qinit = [(0.5*(-0.436332)) + (0.5*(0.436332))]*dofs

amin = bmin = -1.5
amax = bmax = 1.5
ainit = [[0] * nf] * dofs
binit = [[0] * nf] * dofs
abnds = []
bbnds = []
qbnds = [None] * dofs
for i in range(0, dofs):
    qbnds[i] = (-0.5, 0.5)
    for j in range(0, nf):
        abnds.append((-1.5, 1.5))
        bbnds.append((-1.5, 1.5))
        ainit[i][j] = binit[i][j] = 0.3
wbnds = [(wf_min, wf_max)]

bnds = [wbnds, qbnds, abnds, bbnds]
bnds = [bnd for sublist in bnds for bnd in sublist]
print(bnds)

x0=[]
x0 = np.append([wf_init], qinit)
x0 = np.append(x0, np.ravel(ainit))
x0 = np.append(x0, np.ravel(binit))
x0 =[0.207237,0.000023247, -0.309111, -0.14981845, -0.2037325, -0.27274792,-0.5996635, 0.915887509, 0.3365387, -0.234871919,0.777777, -0.1020471, -0.035883, 0.9934340,1.0063, -0.8417213, -0.54149, 0.944568, 0.56095, 0.46266, 0.03673, -0.57127, 0.943396, -0.32849, 0.109994023, 0.70574,-0.381049, 0.832930, 1.0313, 0.8106638,-0.159298, 0.029010, 1.11428, 0.3471255, 0.23880, 1.01124, -0.01446305, 0.88860, 0.626896, -0.504415, -0.473716, -0.64263, -0.58491, -0.39494, 0.32540, -0.1499524]
print(len(x0),len(bnds))
def vecToParams(x):
        # convert vector of all solution variables to separate parameter variables
        nf = [3]*dofs
        wf = x[0]
        q = x[1:dofs+1]
        ab_len = dofs*nf[0]
        a = np.array(np.split(np.array(x[dofs+1:dofs+1+ab_len]), dofs))
        b = np.array(np.split(np.array(x[dofs+1+ab_len:dofs+1+ab_len*2]),dofs))
        return wf,q,a, b


[(0.01, 2.0), (-0.5, 0.5), (-0.5, 0.5), (-0.5, 0.5), (-0.5, 0.5), (-0.5, 0.5), (-1.5, 1.5), (-1.5, 1.5), (-1.5, 1.5), (-1.5, 1.5), (-1.5, 1.5), (-1.5, 1.5), (-1.5, 1.5), (-1.5, 1.5), (-1.5, 1.5), (-1.5, 1.5), (-1.5, 1.5), (-1.5, 1.5), (-1.5, 1.5), (-1.5, 1.5), (-1.5, 1.5), (-1.5, 1.5), (-1.5, 1.5), (-1.5, 1.5), (-1.5, 1.5), (-1.5, 1.5), (-1.5, 1.5), (-1.5, 1.5), (-1.5, 1.5), (-1.5, 1.5), (-1.5, 1.5), (-1.5, 1.5), (-1.5, 1.5), (-1.5, 1.5), (-1.5, 1.5), (-1.5, 1.5), (-1.5, 1.5), (-1.5, 1.5), (-1.5, 1.5), (-1.5, 1.5), (-1.5, 1.5), (-1.5, 1.5), (-1.5, 1.5), (-1.5, 1.5), (-1.5, 1.5), (-1.5, 1.5)]
(46, 46)


In [10]:
from identification.model import Model
import iDynTree; iDynTree.init_helpers(); iDynTree.init_numpy_helpers()

import yaml


with open("/home/admin/rnd_test/FloBaRoID/configs/youbot_test.yaml", 'r') as stream:
    try:
        config = yaml.load(stream)
    except yaml.YAMLError as exc:
        print(exc)

config['urdf'] = "/home/admin/rnd_test/FloBaRoID/model/youbot_arm.urdf"
model = Model(config, config['urdf'])

def objectiveFunc(x):
    wf, q, a, b = vecToParams(x)
    print(wf, q, a, b)
    nf = [4] * dofs
    trajectory = PulsedTrajectory(dofs, use_deg=0)
    trajectory.initWithParams(a, b, q, nf, wf)
    trajectory_data = simulateTrajectory(trajectory, config)
    
    return(np.linalg.cond(model.YBase))

def cont_1(x):
    wf, q, a, b = vecToParams(x)
    nf = [4] * dofs
    limits_low = [0.0, 0.0, -5.18363, 0.0, 0.0]
    trajectory = PulsedTrajectory(dofs, use_deg=0)
    trajectory.initWithParams(a, b, q, nf, wf)
    trajectory_data = simulateTrajectory(trajectory, config)
    
    for n in range(0, dofs):
        g_low = (limits_low[n]) - np.min(trajectory_data['positions'][:, n])
    
    return g_low

def cont_2(x):
    wf, q, a, b = vecToParams(x)
    nf = [4] * dofs
    limits_upper=[5.89921,2.70526,0,3.57792,5.7595] 
    trajectory = PulsedTrajectory(dofs, use_deg=0)
    trajectory.initWithParams(a, b, q, nf, wf)
    trajectory_data = simulateTrajectory(trajectory, config)
    
    for n in range(0, dofs):
        g_upper = np.max(trajectory_data['positions'][:, n]) - limits_upper[n]
    
    return g_upper

def cont_3(x):
    wf, q, a, b = vecToParams(x)
    nf = [4] * dofs
    limits_vel=[1.57079632679]*5 
    trajectory = PulsedTrajectory(dofs, use_deg=0)
    trajectory.initWithParams(a, b, q, nf, wf)
    trajectory_data = simulateTrajectory(trajectory, config)
    
    for n in range(0, dofs):
        g_vel = np.max(np.abs(trajectory_data['velocities'][:, n])) - limits_vel[n]
    
    return g_vel


con1 = {'type': 'ineq', 'fun': cont_1} 
con2 = {'type': 'ineq', 'fun': cont_2}
con3 = {'type': 'ineq', 'fun': cont_3}
cons = ([con1,con2,con3])    


    


loaded model /home/admin/rnd_test/FloBaRoID/model/youbot_arm.urdf
# DOFs: 5
Joints: ['arm_joint_1', 'arm_joint_2', 'arm_joint_3', 'arm_joint_4', 'arm_joint_5']
# regressor outputs: 5
# links: 6 (+ 0 fake)
{0: 'arm_link_0', 1: 'arm_link_1', 2: 'arm_link_2', 3: 'arm_link_3', 4: 'arm_link_4', 5: 'arm_link_5'}
# params: 60 (60 will be identified)
loaded random structural regressor from /home/admin/rnd_test/FloBaRoID/model/youbot_arm.urdf.regressor.npz


In [12]:
from scipy.optimize import minimize, rosen, rosen_der
res = minimize(objectiveFunc,x0, method='SLSQP', bounds=bnds, constraints=cons, options={"disp": True})

IndexError: index 3 is out of bounds for axis 0 with size 3